In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=5e97b53e87a79d9f3c552f2cf8f05a6c925682c625e24f2bd00b2a1feea6c191
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.6MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F24 '''
def F24(X):
    f = bn.F24()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.0257 ,random_state=0 , l1_ratio=0.9795, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 8.05 , C = 787.74 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_244000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_24_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(0, 10):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=187818, Fri Apr 17 20:34:17 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 9.423696201394499e+03 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 9.390525055399254e+03 1.0e+00 2.39e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9431.05379942] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9452.2887858] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 9.390044447053440e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   31    589 9.145393897952405e+03 1.1e+00 1.92e+00  2e+00  2e+00 0:03.5
   67   1273 8.991967418448652e+03 1.1e+00 1.61e+00  2e+00  2e+00 0:07.5
  100   1900 8.536873532465894e+03 1.2e+00 1.21e+00  1e+00  1e+00 0:11.2
  154   2926 7.313535432561574e+03 1.2e+00 7.20e-01  7e-01  7e-01 0:17.2
  200   3800 6.652062938144409e+03 1.2e+00 4.35e-01  4e-01  4e-01 0:22.4
  268   5092 6.136606919103750e+03 1.2e+00 2.14e-01  2e-01  2e-01 0:30.4
  300   5700 6.035087677957082e+03 1.2e+00 1.46e-01  1e-01  1e-01 0:34.0
  392   7448 5.978020613307061e+03 1.2e+00 5.58e-02  5e-02  6e-02 0:44.1
  400   7600 5.975293370712418e+03 1.2e+00 5.09e-02  5e-02  5e-02 0:45.0
  500   9500 5.966916588659178e+03 1.2e+00 1.64e-02  2e-02  2e-02 0:56.0
  600  11400 5.966115381499329e+03 1.2e+00 5.62e-03  6e-03  6e-03 1:06.9
  700  13300 5.966012802304154e+03 1.2e+00 1.91e-03  2e-03  2e-03 1:18.0
  800  15200 5.966002108867601e+03 1.2e+00 6.07e-04

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9477.91441782] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9716.80156572] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 9.305811955527755e+03 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 8.753831784147584e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


...

    3     57 9.383503657381878e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   26    494 9.149829684094046e+03 1.0e+00 1.94e+00  2e+00  2e+00 0:03.5
   56   1064 8.989824737141824e+03 1.1e+00 1.65e+00  2e+00  2e+00 0:07.6
   88   1672 8.710821047698770e+03 1.2e+00 1.38e+00  1e+00  1e+00 0:12.6
  100   1900 8.584329796615664e+03 1.2e+00 1.24e+00  1e+00  1e+00 0:14.4
  151   2869 7.356912591131675e+03 1.2e+00 7.29e-01  7e-01  7e-01 0:21.4
  200   3800 6.546217960961813e+03 1.2e+00 4.18e-01  4e-01  4e-01 0:27.8
  270   5130 6.109202725015608e+03 1.2e+00 1.98e-01  2e-01  2e-01 0:36.9
  300   5700 6.063336616232143e+03 1.2e+00 1.48e-01  1e-01  1e-01 0:40.8
  381   7239 5.982125007757179e+03 1.2e+00 6.79e-02  7e-02  7e-02 0:51.8
  400   7600 5.977243335399370e+03 1.2e+00 5.42e-02  5e-02  5e-02 0:54.3
  500   9500 5.967060507419868e+03 1.2e+00 1.72e-02  2e-02  2e-02 1:07.2
  600  11400 5.966132827006598e+03 1.2e+00 5.93e-03  6e-03  6e-03 1:20.1
  700  13300 5.966012794118534e+03 1.2e+00 1.93e-03

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9477.91441782] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9758.05834563] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 9.274301699625827e+03 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 9.298780958298368e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10029.58113177] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   34    646 7.262335633656987e+03 1.1e+00 1.88e+00  2e+00  2e+00 0:03.4
   75   1425 6.037036668006846e+03 1.1e+00 1.50e+00  1e+00  2e+00 0:07.4
  100   1900 4.458143523568903e+03 1.2e+00 1.36e+00  1e+00  1e+00 0:09.9
  161   3059 3.273674983051531e+03 1.2e+00 8.68e-01  9e-01  9e-01 0:15.9
  200   3800 2.772609514530268e+03 1.2e+00 5.75e-01  6e-01  6e-01 0:19.7
  281   5339 2.272797535070037e+03 1.2e+00 2.73e-01  3e-01  3e-01 0:27.7
  300   5700 2.221919868559122e+03 1.2e+00 2.23e-01  2e-01  2e-01 0:29.6
  386   7334 2.099836132170386e+03 1.3e+00 1.17e-01  1e-01  1e-01 0:39.6
  400   7600 2.095231911749246e+03 1.3e+00 1.03e-01  1e-01  1e-01 0:42.3
  500   9500 2.070609791252797e+03 1.3e+00 4.37e-02  4e-02  4e-02 0:52.6
  600  11400 2.066631355812128e+03 1.3e+00 1.91e-02  2e-02  2e-02 1:02.5
  700  13300 2.065577725903712e+03 1.3e+00 9.44e-03  9e-03  1e-02 1:12.4
  800  15200 2.065385211453915e+03 1.3e+00 4.22e-03  4e-03  4e-03 1:22.3
  900  17100 2.065334916783578e+03 1.3e+00 2.08e-03

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F24_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F24(X_Values.iloc[i,:200])
    SVM_Fun [i] = F24(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F24(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([2661.80688178]), count=array([1])) 0.0004001609794248252
SVM
ModeResult(mode=array([9879.92847124]), count=array([1])) 589.5742783713262
ELN
ModeResult(mode=array([10437.83524057]), count=array([1])) 0.00029461004696917757
